Import Required Libraries

In [ ]:
import tensorflow as tf
from glob import glob
import numpy as np
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from keras.optimizers import Adam
from tensorflow.keras.metrics import *
import kagglehub
from glob import glob
import os

Download Dataset from KaggleHub

In [ ]:
dataset_dir = kagglehub.dataset_download('aryashah2k/breast-ultrasound-images-dataset')
print("Dataset downloaded to:", dataset_dir)

Load Dataset Paths

In [ ]:
dataset_dir = r"C:\Users\info\.cache\kagglehub\datasets\aryashah2k\breast-ultrasound-images-dataset\versions\1\Dataset_BUSI_with_GT"
paths = glob(os.path.join(dataset_dir, "*", "*"))

Check Image and Mask Counts

In [ ]:
dataset_dir = "C:/Users/info/.cache/kagglehub/datasets/aryashah2k/breast-ultrasound-images-dataset/versions/1/Dataset_BUSI_with_GT"
paths = glob(os.path.join(dataset_dir, "*", "*"))

print('\033[92m')
print(f"'normal' class has {len([i for i in paths if 'normal' in i and 'mask' not in i])} images and {len([i for i in paths if 'normal' in i and 'mask' in i])} masks.")
print(f"'benign' class has {len([i for i in paths if 'benign' in i and 'mask' not in i])} images and {len([i for i in paths if 'benign' in i and 'mask' in i])} masks.")
print(f"'malignant' class has {len([i for i in paths if 'malignant' in i and 'mask' not in i])} images and {len([i for i in paths if 'malignant' in i and 'mask' in i])} masks.")
print(f"\nTotal: {len([i for i in paths if 'mask' not in i])} images and {len([i for i in paths if 'mask' in i])} masks.")


View Sample Paths

In [ ]:
sample_paths = sorted(glob(os.path.join(dataset_dir, "benign", "*")))[4:7]
for path in sample_paths:
    print(path)

 Define Helper Functions for Loading Data

In [ ]:
def load_image(path, size):
    image = cv2.imread(path)
    image = cv2.resize(image, (size, size))
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    image = image / 255.0
    return image

In [ ]:
def load_data(root_path_glob, size):
    images = []
    masks = []
    x = 0
    for path in sorted(glob(root_path_glob)):
        img = load_image(path, size)
        if 'mask' in path:
            if x:
                masks[-1] += img
                masks[-1] = np.array(masks[-1] > 0.5, dtype='float64')
            else:
                masks.append(img)
                x = 1
        else:
            images.append(img)
            x = 0
    return np.array(images), np.array(masks)

Load Images and Masks

In [ ]:
size = 128
dataset_glob_path = "C:/Users/info/.cache/kagglehub/datasets/aryashah2k/breast-ultrasound-images-dataset/versions/1/Dataset_BUSI_with_GT/*/*"
X, y = load_data(root_path_glob=dataset_glob_path, size=size)


Visualize Random Sample - Normal Class

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(10,5))
i = np.random.randint(647,780)
ax[0].imshow(X[i], cmap='gray')
ax[0].set_title('Image')
ax[1].imshow(y[i], cmap='gray')
ax[1].set_title('Mask')
ax[2].imshow(X[i], cmap='gray')
ax[2].imshow(tf.squeeze(y[i]), alpha=0.5, cmap='jet')
ax[2].set_title('Union')
fig.suptitle('Normal class', fontsize=16)
plt.show()

Visualize Random Sample - Benign Class

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(10,5))
i = np.random.randint(437)
ax[0].imshow(X[i], cmap='gray')
ax[0].set_title('Image')
ax[1].imshow(y[i], cmap='gray')
ax[1].set_title('Mask')
ax[2].imshow(X[i], cmap='gray')
ax[2].set_title('Union')
ax[2].imshow(tf.squeeze(y[i]), alpha=0.5, cmap='jet')
fig.suptitle('Benign class', fontsize=16)
plt.show()

Visualize Random Sample - Malignant Class

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(10,5))
i = np.random.randint(437,647)
ax[0].imshow(X[i], cmap='gray')
ax[0].set_title('Image')
ax[1].imshow(y[i], cmap='gray')
ax[1].set_title('Mask')
ax[2].imshow(X[i], cmap='gray')
ax[2].imshow(tf.squeeze(y[i]), alpha=0.5, cmap='jet')
ax[2].set_title('Union')
fig.suptitle('Malignant class', fontsize=16)
plt.show()

Visualize Combined Masks for Each Class

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(10,5))
ax[0].imshow(sum(y[647:]), cmap='gray')
ax[0].set_title('Normal')
ax[1].imshow(sum(y[:437]), cmap='gray')
ax[1].set_title('Benign')
ax[2].imshow(sum(y[437:647]), cmap='gray')
ax[2].set_title('Malignant')
plt.show()

Drop Normal Class and Prepare Data

In [ ]:
X = X[:647]
y = y[:647]
print(f"X shape: {X.shape}     |  y shape: {y.shape}")
X = np.expand_dims(X, -1)
y = np.expand_dims(y, -1)
print(f"\nX shape: {X.shape}  |  y shape: {y.shape}")

Split into Train and Test Sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
print('\033[92m')
print('X_train shape:',X_train.shape)
print('y_train shape:',y_train.shape)
print('X_test shape:',X_test.shape)
print('y_test shape:',y_test.shape)

Define Convolution Block

In [ ]:
def conv_block(input, num_filters):
    conv = Conv2D(num_filters, (3, 3), activation="relu", padding="same", kernel_initializer='he_normal')(input)
    conv = Conv2D(num_filters, (3, 3), activation="relu", padding="same", kernel_initializer='he_normal')(conv)
    return conv

Define Encoder Block

In [ ]:
def encoder_block(input, num_filters):
    conv = conv_block(input, num_filters)
    pool = MaxPooling2D((2, 2))(conv)
    return conv, pool

Define Decoder Block

In [ ]:
def decoder_block(input, skip_features, num_filters):
    uconv = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    con = concatenate([uconv, skip_features])
    conv = conv_block(con, num_filters)
    return conv

Build U-Net Model

In [ ]:
def build_model(input_shape):
    input_layer = Input(input_shape)
    s1, p1 = encoder_block(input_layer, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)
    b1 = conv_block(p4, 1024)
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)
    output_layer = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)
    model = Model(input_layer, output_layer, name="U-Net")
    return model

model = build_model(input_shape=(size, size, 1))
model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])


Plot U-Net Model Architecture

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

Show Model Summary

In [ ]:
model.summary()

Train the Model

In [ ]:
history = model.fit(X_train, y_train, epochs = 100, validation_data = (X_test,y_test))

Traning Loss and Accuracy Plot

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(12, 4))

ax[0].plot(history.epoch, history.history["loss"], label="Train Loss", color='blue')
ax[0].plot(history.epoch, history.history["val_loss"], label="Validation Loss", color='orange')
ax[0].set_title("Loss Over Epochs")
ax[0].set_xlabel("Epoch")
ax[0].set_ylabel("Loss")
ax[0].legend()
ax[0].grid(True)

ax[1].plot(history.epoch, history.history["accuracy"], label="Train Accuracy", color='green')
ax[1].plot(history.epoch, history.history["val_accuracy"], label="Validation Accuracy", color='red')
ax[1].set_title("Accuracy Over Epochs")
ax[1].set_xlabel("Epoch")
ax[1].set_ylabel("Accuracy")
ax[1].legend()
ax[1].grid(True)

fig.suptitle("Training Progress: Loss and Accuracy", fontsize=16)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])

plt.show()


Save Model Weights

In [ ]:
model.save_weights('model.weights.h5')